In [1]:
%load_ext lab_black

In [114]:
import math
import os
import random
import shutil
import json
import logging
import sys
from collections import defaultdict
from datetime import datetime
from io import TextIOWrapper
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from tqdm import tqdm
from PIL import Image

import utils
import networks

In [27]:
MEAN = torch.tensor((0.485, 0.456, 0.406), dtype=torch.float32).reshape(3, 1, 1)
STD = torch.tensor((0.229, 0.224, 0.225), dtype=torch.float32).reshape(3, 1, 1)

In [28]:
class ImageDataset(Dataset):
    def __init__(self, files, keypoints=None, padding=40):
        super().__init__()
        self.files = files
        self.keypoints = keypoints
        self.padding = padding

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        f = self.files[idx]
        img = imageio.imread(f)
        x = img[102:998, 374:1526]
        x = torch.tensor(x, dtype=torch.float32).permute(2, 0, 1).div(255.0)
        x = (x - MEAN) / STD

        return x

In [29]:
train_imgs = sorted(list(Path("data/ori/train_imgs/").glob("*.jpg")))
test_imgs = sorted(list(Path("data/ori/test_imgs/").glob("*.jpg")))

In [30]:
ds_train = ImageDataset(train_imgs)
ds_test = ImageDataset(test_imgs)

In [31]:
dl_train = DataLoader(ds_train, batch_size=10, num_workers=4, shuffle=False)
dl_test = DataLoader(ds_test, batch_size=10, num_workers=4, shuffle=False)

In [32]:
model = networks.EfficientDet("efficientdet-d7", pretrained=True)
model = model.cuda()
model.eval()
torch.set_grad_enabled(False)
pass

In [33]:
dir_train = Path("data/box_effdet-d7/train")
dir_test = Path("data/box_effdet-d7/test")
dir_train.mkdir(parents=True, exist_ok=True)
dir_test.mkdir(parents=True, exist_ok=True)

In [ ]:
i = 0
with tqdm(total=len(dl_train.dataset), ncols=100, file=sys.stdout) as t:
    for xs in dl_train:
        xs_ = xs.cuda()
        outs_ = model(xs_)
        for x, p in zip(xs, outs_):
            box = p["rois"][p["class_ids"] == 0][p["scores"][p["class_ids"] == 0].argmax()].astype(np.int64).tolist()
            out = ((x * STD + MEAN) * 255.).permute(1,2,0).type(torch.uint8).numpy()
            out = np.array(Image.fromarray(out))
            out = cv2.rectangle(out, (box[0], box[1]), (box[2], box[3]), (255, 0,0), 2)
            imageio.imwrite(dir_train/train_imgs[i].name, out)
            
            i += 1
            t.update()

 37%|█████████████████████▋                                     | 1539/4195 [02:57<03:58, 11.12it/s]